<a href="https://colab.research.google.com/github/erika0915/25-machine-learning/blob/main/HA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-On Assignment 2

2-1. Support vector machines with Iris dataset

## After finishing writing the code for each problem, submit your ipynb file (i.e., this file) to Cybercampus by **10/24 Friday 23:59**

### Submitters are as follows.
 *Note that hands-on assignments can be done collaboratively (**up to 2 students**)*

    Name: 최수희
    Student ID: 2271104



---


## 2-1. Support vector machines with Iris dataset


---



Download the dataset


*   data source: scikit-learn library



In [4]:
from sklearn.datasets import load_iris

# Scikit-learn 라이브러리에 내장된 Iris 데이터셋 불러오기
iris_dataset = load_iris()

## Goal: Train Support vector machines to predict y (species of flower) based on provided features (SepalLength, SepalWidth, PetalLength, PetalWidth)
❗❗READ CAREFULLY ⬇

*   Use Scikit-learn library directly
*   TO-DO:
  1. Split the dataset into training and testing sets (training:testing=75:25)
  2. Use "StandardScaler" function (provided in HA1_part1) to map all points to have mean=0 and std=1
  3. First, train and test SVM using linear kernel
    *   Report the accuracy
    *   After that, by using 5-fold cross validation, again report accuracies from each fold and the mean accuracy after taking the mean on five accuracies.
  4. Second, train and test SVM using RBF kernel
    *   use C=1, gamma=0.01
    *   Report the accuracy
    *   After that, by using 5-fold cross validation, again report accuracies from each fold and the mean accuracy after taking the mean on five accuracies.
  5. Hyperparemter tuning by using Grid Search (i.e., from sklearn.model_selection import GridSearchCV)
    *   Parameter search space must include three kernels (['rbf', 'poly', 'sigmoid'], C ([0.1,1, 10, 100]) and gamma ([1,0.1,0.01,0.001]))
    *   Report best parameters and accuracy based on them
  6. Extend your discussion on why chosen kernel works more effectively than others and whether hyperparmeter tuning helps perfomrance improvement.

#### 1. Split the dataset

In [5]:
from sklearn.model_selection import train_test_split

X = iris_dataset.data
y = iris_dataset.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [6]:
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

X_train shape: (112, 4)
X_test shape: (38, 4)
y_train shape: (112,)
y_test shape: (38,)


#### 2. Standardization

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#### 3. SVM (linear)
- 테스트 데이터로 정확도 측정
- 5-fold cross-validation 실행

In [13]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np

# 선형 커널을 사용하는 SVM 모델 생성
linear_svm = SVC(kernel='linear')

linear_svm.fit(X_train_scaled, y_train)
y_pred = linear_svm.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"Linear SVM Accuracy on the test set : {accuracy: .4f}")


Linear SVM Accuracy on the test set :  0.9737


In [14]:
scores = cross_val_score(linear_svm, X_train_scaled, y_train, cv=5)

print("Cross validation Scores :", scores)
print(f"Mean Accuracy from Cross validation :{np.mean(scores):.4f}")

Cross validation Scores : [0.95652174 1.         0.81818182 1.         0.95454545]
Mean Accuracy from Cross validation :0.9458


#### 4. SVM (RBF 커널)
- 테스트 데이터로 정확도 측정
- 5-fold cross-validation 실행

In [15]:
# RBF 커널을 사용하는 SVM 생성
rbf_svm = SVC(kernel='rbf', C=1, gamma=0.01)

rbf_svm.fit(X_train_scaled, y_train)
y_pred_rbf = rbf_svm.predict(X_test_scaled)

accuracy_rbf = accuracy_score(y_test, y_pred_rbf)
print(f"RBF SVM Accuracy on the test set : {accuracy_rbf:.4f}")

RBF SVM Accuracy on the test set : 0.9737


In [16]:
scores = cross_val_score(rbf_svm, X_train_scaled, y_train, cv=5)

print("Cross validation Scores :", scores)
print(f"Mean Accuracy from Cross validation :{np.mean(scores):.4f}")

Cross validation Scores : [0.82608696 0.82608696 0.86363636 0.95454545 0.86363636]
Mean Accuracy from Cross validation :0.8668


#### 5. Grid Search
- 테스트할 하이퍼파라미터 후보들을 목록으로 만든다.
- `GridSearchCV` 도구를 사용해 이 모든 조합에 대해 5-fold cross-validation을 수행
- 찾아낸 최고의 성능을 보인 파라미터 조합과 그때의 교차 검증 평균 정확도 출력


In [18]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'kernel' : ['rbf', 'poly', 'sigmoid'],
    'C' : [0.1, 1, 10, 100],
    'gamma' : [1, 0.1, 0.01, 0.001]
}

# 기본 SVM 모델
svm = SVC()

# GridSearchCV 생성
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

print("Best parameters found: ", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

Best parameters found:  {'C': 100, 'gamma': 0.1, 'kernel': 'rbf'}
Best cross-validation score: 0.9549


#### 6. 최종 분석
 - 선형 SVM 평균 정확도 : 0.9458
 - 임의 설정 RBF SVM 평균 정확도 : 0.8668
 - Grid Search 최적 RBF SVM 평균 정확도 : 0.9549

 Grid Search를 통해 찾아낸, `{'C':100, 'gamma':0.1, 'kernel': 'rbf'}` 조합이 단순 선형 SVM보다도, 그리고 임의로 설정했던 RBF SVM보다도 높은 성능을 보여줬습니다. 이는 하이퍼파라미터 튜닝이 모델의 성능을 끌어올리는 데 얼마나 중요한지 명확하게 보여주는 결과입니다.

 <br>

 1. 왜 최종 선택된 커널이 효과적이었나 ?

 Iris 데이터셋은 전반적으로 선형 분리가 잘 되는 특성을 가지고 있어 선형 커널만으로도 높은 성능을 보였습니다. 하지만 RBF 커널은 데이터를 고차원 공간으로 매핑하여 비선형적인 결정 경계를 만들 수 있으므로, 두 품종이 미세하게 겹치는 영역을 선형 커널보다 더 유연하고 정교하게 구분해낼 수 있었습니다. 최적화된  C와 gamma 값을 통해 RBF 커널이 이 데이터의 특성을 가장 잘 학습했다고 결론 내릴 수 있습니다.

 2. 하이퍼파라미터 튜닝이 모델 성능 향상에 도움이 되었는가 ?

임의로 설정한 RBF 모델의 평균 정확도는 약 86.7%에 불과했습니다. 하지만 Grid Search를 통해 최적의 하이퍼파라미터를 탐색한 결과, RBF 커널의 성능은 95.5%까지 향상되었습니다. 이는 단순히 기본값을 사용하거나 임의로 파라미터를 설정하는 것보다 체계적인 탐색 과정을 통해 데이터에 가장 적합한 모델을 찾는 것이 왜 중요한지를 명확히 보여줍니다.
